# Import Packages

In [1]:
import numpy as np
import pandas as pd
import glob
import os

# Define Dynamic Parameters

In [2]:
trend_days = 3
trend_file_name = 'GoogleTrendsAdjusted.csv'
trends_variant = 'No'

In [3]:
# Parameters
trend_days = 7
trend_file_name = "GoogleTrendsAdjusted.csv"
trends_variant = ""


# Import CSVs

In [4]:
# Current Path
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '/financial_data'
print(path)

csvfiles = glob.glob(path + "/*.csv")

# Creating a dataframe for each CSV file
dfs = [pd.read_csv(file) for file in csvfiles]

C:\FYP - Luke Bezzina\Code\preprocessingHistoricalData/financial_data


# Import Google Trends Adjusted dataset

In [5]:
# Current Path
trends_path = ROOT_DIR + '/trends/'
print(trends_path)

# Creating a dataframe for trends
google_df = pd.read_csv(trends_path + trend_file_name)

C:\FYP - Luke Bezzina\Code\preprocessingHistoricalData/trends/


# New Dataframe

In [6]:
processed_df_coll = {}

# Pre-Process

In [7]:
prev_row = None
# Up to 0.2% deviation from close price from day before is considered Neutral movement
neutral_percentage = 0.002

for df in dfs:
    processed_df = pd.DataFrame(columns = ['Date', 'OpenPrice', 'ClosePrice', 'Trend',
                                           'VolumeTrend', 'Volatility', 'PriceDirection'])

    # Creating additional feature columns for each dataframe
    iterables = df.itertuples(index=True, name='Pandas')
    n = df.columns.get_loc('Name') # Name column in df
    df_name = df.iat[1, n] # Name of equity/etf

    print(df_name)

    # Iterating all rows in dataframe
    # Starting from trend_days - to allow for trends going back to trend_days days
    # Ending at end day - 2 to allow for day + 2 classification label
    for i in range(trend_days, len(df) - 2 ):
        open = df.columns.get_loc('Open')
        close = df.columns.get_loc('Close')
        volume = df.columns.get_loc('Volume')
        df_date = df.columns.get_loc('Date')

        # Prices per row
        open_price = df.iat[i-1, open]
        close_price = df.iat[i-1, close]

        # Price Trend Handling
        price_today = df.iat[i + 0, close]
        price_tomorrow = df.iat[i + 1, close]
        price_2days_after = df.iat[i + 2, close]

        close_price_3d = (price_today + price_tomorrow + price_2days_after)/3
        prev_close_price = df.iat[i - 1, close]
        price_change = close_price_3d - prev_close_price

        if price_change > (close_price_3d * neutral_percentage):
            direction = 'Positive'
        elif price_change < 0 and abs(price_change) > (close_price_3d * neutral_percentage):
            direction = 'Negative'
        else:
            direction = 'Neutral'

        # Price Difference Handling
        pds = {}
        initial_close = df.iat[i - (trend_days + 1), close]

        for d in range(1, trend_days + 1):
            key = "d"+ str(d) +"_pd"
            value = df.iat[i - d, close] - initial_close
            pds[key] = value

        # Volume Difference Handling
        vpds = {}
        initial_vol = df.iat[i - (trend_days + 1), volume] + 1

        for vd in range(1, trend_days + 1):
            key = "vd"+ str(vd) +"_pd"
            value = (df.iat[i - vd, volume] / initial_vol)
            vpds[key] = value

        date_ts = df.iat[i,df_date]

        # volatility of prices window
        volatility = np.std(list(pds.values()))

        # finding trends of close and volume in terms of gradient (slope)
        polyfit_x_axis = list(range(0, trend_days))
        test_list = list(pds.values())
        slope, intercept = np.polyfit(polyfit_x_axis, list(pds.values()), 1)
        slopeVol, interceptVol = np.polyfit(polyfit_x_axis, list(vpds.values()), 1)

        new_row = {'Date':date_ts, 'OpenPrice':open_price, 'ClosePrice':close_price,
                   'Trend':slope, 'VolumeTrend':slopeVol, 'Volatility':volatility,
                   'PriceDirection':direction}

        #append row to the dataframe
        processed_df = processed_df.append(new_row, ignore_index=True)

    #pd.concat([processed_df, google_df], axis=1)
    processed_df = pd.merge(processed_df,google_df,left_on='Date',right_on='Date')
    # Re-sorting PriceDirection column
    label = processed_df.pop('PriceDirection')
    processed_df['PriceDirection'] = label

    print(processed_df)
    processed_df_coll[df_name] = processed_df


AAL


           Date  OpenPrice  ClosePrice  ...  variant  pfizer  PriceDirection
0    2017-01-12      49.29       48.64  ...     6.93    1.77        Negative
1    2017-01-13      48.67       48.10  ...     8.69    1.50        Negative
2    2017-01-17      48.16       47.65  ...     4.16    2.64        Negative
3    2017-01-18      47.56       46.75  ...     3.44    1.77        Positive
4    2017-01-19      46.72       47.64  ...     6.80    2.10        Negative
..          ...        ...         ...  ...      ...     ...             ...
993  2020-12-22      15.70       16.10  ...    17.11   20.16        Negative
994  2020-12-23      16.35       15.48  ...    10.03   20.58        Positive
995  2020-12-24      15.52       15.89  ...     8.26   13.86         Neutral
996  2020-12-28      16.04       15.66  ...    15.00   11.55        Positive
997  2020-12-29      15.96       16.06  ...    36.00   12.87        Negative

[998 rows x 25 columns]
AAPL


           Date  OpenPrice  ClosePrice  ...  variant  pfizer  PriceDirection
0    2017-01-12      29.68       29.94  ...     6.93    1.77        Negative
1    2017-01-13      29.72       29.81  ...     8.69    1.50        Positive
2    2017-01-17      29.78       29.76  ...     4.16    2.64        Positive
3    2017-01-18      29.59       30.00  ...     3.44    1.77         Neutral
4    2017-01-19      30.00       30.00  ...     6.80    2.10         Neutral
..          ...        ...         ...  ...      ...     ...             ...
993  2020-12-22     125.02      128.23  ...    17.11   20.16        Positive
994  2020-12-23     131.61      131.88  ...    10.03   20.58        Positive
995  2020-12-24     132.16      130.96  ...     8.26   13.86        Positive
996  2020-12-28     131.32      131.97  ...    15.00   11.55        Positive
997  2020-12-29     133.99      136.69  ...    36.00   12.87        Negative

[998 rows x 25 columns]
AMT


           Date  OpenPrice  ClosePrice  ...  variant  pfizer  PriceDirection
0    2017-01-12     103.48      103.83  ...     6.93    1.77        Positive
1    2017-01-13     103.69      104.20  ...     8.69    1.50        Positive
2    2017-01-17     104.22      103.45  ...     4.16    2.64        Positive
3    2017-01-18     103.58      105.17  ...     3.44    1.77        Negative
4    2017-01-19     105.26      105.67  ...     6.80    2.10        Negative
..          ...        ...         ...  ...      ...     ...             ...
994  2020-12-22     219.13      219.78  ...    17.11   20.16        Negative
995  2020-12-23     219.07      220.49  ...    10.03   20.58        Negative
996  2020-12-24     221.50      217.40  ...     8.26   13.86        Positive
997  2020-12-28     216.44      218.65  ...    15.00   11.55        Positive
998  2020-12-29     218.87      221.07  ...    36.00   12.87        Positive

[999 rows x 25 columns]
AVY


           Date  OpenPrice  ClosePrice  ...  variant  pfizer  PriceDirection
0    2017-01-12      72.48       72.70  ...     6.93    1.77        Negative
1    2017-01-13      72.46       72.13  ...     8.69    1.50        Positive
2    2017-01-17      72.43       72.15  ...     4.16    2.64        Positive
3    2017-01-18      72.16       72.45  ...     3.44    1.77        Positive
4    2017-01-19      72.44       73.17  ...     6.80    2.10        Negative
..          ...        ...         ...  ...      ...     ...             ...
993  2020-12-22     150.69      152.08  ...    17.11   20.16        Positive
994  2020-12-23     151.37      151.76  ...    10.03   20.58        Positive
995  2020-12-24     152.67      152.71  ...     8.26   13.86        Positive
996  2020-12-28     153.57      153.40  ...    15.00   11.55         Neutral
997  2020-12-29     155.00      153.94  ...    36.00   12.87         Neutral

[998 rows x 25 columns]
AWK


...

           Date  OpenPrice  ClosePrice  ...  variant  pfizer  PriceDirection
0    2017-01-12      90.65       90.44  ...     6.93    1.77        Positive
1    2017-01-13      90.45       90.51  ...     8.69    1.50        Positive
2    2017-01-17      90.42       90.40  ...     4.16    2.64        Positive
3    2017-01-18      90.63       92.36  ...     3.44    1.77        Positive
4    2017-01-19      92.60       93.49  ...     6.80    2.10        Positive
..          ...        ...         ...  ...      ...     ...             ...
993  2020-12-22      84.53       84.17  ...    17.11   20.16        Negative
994  2020-12-23      82.53       82.19  ...    10.03   20.58        Positive
995  2020-12-24      82.45       82.20  ...     8.26   13.86         Neutral
996  2020-12-28      82.16       82.51  ...    15.00   11.55        Negative
997  2020-12-29      82.81       82.49  ...    36.00   12.87        Negative

[998 rows x 25 columns]
S&P500


           Date  OpenPrice  ClosePrice  ...  variant  pfizer  PriceDirection
0    2017-01-12    2268.60     2275.32  ...     6.93    1.77         Neutral
1    2017-01-13    2271.14     2270.44  ...     8.69    1.50         Neutral
2    2017-01-17    2272.74     2274.64  ...     4.16    2.64        Negative
3    2017-01-18    2269.14     2267.89  ...     3.44    1.77         Neutral
4    2017-01-19    2269.14     2271.89  ...     6.80    2.10        Negative
..          ...        ...         ...  ...      ...     ...             ...
993  2020-12-22    3684.28     3694.92  ...    17.11   20.16         Neutral
994  2020-12-23    3698.08     3687.26  ...    10.03   20.58        Positive
995  2020-12-24    3693.42     3690.01  ...     8.26   13.86        Positive
996  2020-12-28    3694.03     3703.06  ...    15.00   11.55        Positive
997  2020-12-29    3723.03     3735.36  ...    36.00   12.87         Neutral

[998 rows x 25 columns]
WFC


           Date  OpenPrice  ClosePrice  ...  variant  pfizer  PriceDirection
0    2017-01-12      54.60       54.80  ...     6.93    1.77        Negative
1    2017-01-13      54.65       54.50  ...     8.69    1.50         Neutral
2    2017-01-17      55.11       55.31  ...     4.16    2.64        Negative
3    2017-01-18      54.80       53.78  ...     3.44    1.77        Positive
4    2017-01-19      53.78       54.27  ...     6.80    2.10        Positive
..          ...        ...         ...  ...      ...     ...             ...
993  2020-12-22      29.50       29.55  ...    17.11   20.16        Positive
994  2020-12-23      29.65       28.96  ...    10.03   20.58        Positive
995  2020-12-24      29.29       30.35  ...     8.26   13.86        Negative
996  2020-12-28      30.29       29.84  ...    15.00   11.55         Neutral
997  2020-12-29      29.94       29.93  ...    36.00   12.87         Neutral

[998 rows x 25 columns]
WMT


           Date  OpenPrice  ClosePrice  ...  variant  pfizer  PriceDirection
0    2017-01-12      68.20       68.53  ...     6.93    1.77        Negative
1    2017-01-13      68.36       67.97  ...     8.69    1.50         Neutral
2    2017-01-17      67.98       67.13  ...     4.16    2.64        Positive
3    2017-01-18      68.09       68.42  ...     3.44    1.77        Negative
4    2017-01-19      67.88       68.11  ...     6.80    2.10        Negative
..          ...        ...         ...  ...      ...     ...             ...
993  2020-12-22     145.62      145.97  ...    17.11   20.16        Negative
994  2020-12-23     145.18      144.20  ...    10.03   20.58         Neutral
995  2020-12-24     143.72      143.22  ...     8.26   13.86        Positive
996  2020-12-28     143.54      143.50  ...    15.00   11.55        Positive
997  2020-12-29     143.80      145.22  ...    36.00   12.87        Negative

[998 rows x 25 columns]
XOM


           Date  OpenPrice  ClosePrice  ...  variant  pfizer  PriceDirection
0    2017-01-12      85.90       86.81  ...     6.93    1.77         Neutral
1    2017-01-13      87.27       86.34  ...     8.69    1.50        Positive
2    2017-01-17      86.27       86.35  ...     4.16    2.64        Negative
3    2017-01-18      86.58       87.36  ...     3.44    1.77        Negative
4    2017-01-19      87.06       86.28  ...     6.80    2.10        Negative
..          ...        ...         ...  ...      ...     ...             ...
993  2020-12-22      40.99       41.95  ...    17.11   20.16        Negative
994  2020-12-23      41.63       41.24  ...    10.03   20.58        Positive
995  2020-12-24      41.55       41.77  ...     8.26   13.86        Negative
996  2020-12-28      41.65       41.60  ...    15.00   11.55         Neutral
997  2020-12-29      41.69       41.74  ...    36.00   12.87        Negative

[998 rows x 25 columns]


# Data Export

In [8]:
for key, df in processed_df_coll.items():
    df.to_csv(ROOT_DIR+'\\LSTM\\'+key+"_"+ str(trend_days) + "dtrend_google_trd" + trends_variant + ".csv", index=False)

print("Export Complete!")


Export Complete!
